# This should pull the data, Steam Games Data, from kaggle. After this should clean out the irrelevant or missing data, then import it into PostgreSQL

NOTICE! 
- Ensure that your token for Kaggle is in the API folder
- Ensure that you configure API/PostgreSQL.py with appropriate information, once it is generated by cell 2


In [34]:
import sys
import os
import pandas as pd
sys.path.insert(1,"../API")
import PostgreSQL

APIPATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'API'))
os.environ['KAGGLE_CONFIG_DIR'] = APIPATH

# This is one of the libaries (opendataset) not covered in class that we'll use in this project; this allows us to pull data directly from Kaggle API.
# Make sure to use: "pip install kaggle" in the terminal of your choice, or simply 'pip install kaggle' in a new Jupyter Cell
import kaggle

In [35]:
#Check's for PostgreSQL.py configuration file, if non, it will generate one.
#NOTICE -- Please configure API/PostgreSQL.py before running all code. Ensure it is added to 'gitignore' before running code and pushing to the repository
#ADDITIONALLY -- If the strings get renamed, simply delete 'PostgreSQL.py' and re-run this code.

sqlconfig = os.path.join("..","API","PostgreSQL.py")

if not os.path.exists(sqlconfig):
    with open(sqlconfig, 'w') as file:
        file.write("connectionString = ''\nconnectionStringLocal = ''\nuser = ''\npassword = ''\nhost = ''\nport = ''")
    print(f"Configuration file has been created")
else:
    print(f"Configuration file already exists, importing...")
    


Configuration file already exists, importing...


In [36]:
# Downloading data from Kaggle
# Check directory for if statement to see if data has already been downloaded
checkfordata = os.path.join("..","RawData","games.csv")
# This if loop will check to see if the data has been download, if not it will download and import, if so it will simply import
if not os.path.exists(checkfordata):
    downloadPath = os.path.abspath(os.path.join(os.getcwd(), '..', 'RawData'))
    kaggle.api.dataset_download_files('fronkongames/steam-games-dataset', path=downloadPath, unzip=True)
    # Deleting the .json because we don't need it.
    bye = os.path.join('..','RawData','games.json')
    os.remove(bye)
    print(f"Downloaded dataset and deleted json, proceeding to import...")
    # Loading the data into a data frame
    gamesCSV = os.path.join('..','RawData','games.csv')
    SteamGamesRaw = pd.read_csv(gamesCSV)
    print(f"Dataset has been downloaded and imported!")
else:
    print(f"Dataset already exists, importing...")
    # Loading the data into a data frame
    gamesCSV = os.path.join('..','RawData','games.csv')
    SteamGamesRaw = pd.read_csv(gamesCSV)
    print(f"Dataset has been imported!")

Dataset already exists, importing...
Dataset has been imported!


In [37]:
# Hooray!!! 🥳🥳🥳
SteamGamesRaw.head()

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Average playtime two weeks,Median playtime forever,Median playtime two weeks,Developers,Publishers,Categories,Genres,Tags,Screenshots,Movies
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,0,0,0,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports","Indie,Casual,Sports,Bowling",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie","Indie,Action,Pixel Graphics,2D,Retro,Arcade,Sc...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,0,0,0,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy",NaN,https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,0,0,0,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie","2D Platformer,Atmospheric,Surreal,Mystery,Puzz...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,0,0,0,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie","Indie,Adventure,Nudity,Violent,Sexual Content,...",https://cdn.akamai.steamstatic.com/steam/apps/...,http://cdn.akamai.steamstatic.com/steam/apps/2...


In [38]:
# CLEANING TIME
SteamGamesRaw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85103 entries, 0 to 85102
Data columns (total 39 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   AppID                       85103 non-null  int64  
 1   Name                        85097 non-null  object 
 2   Release date                85103 non-null  object 
 3   Estimated owners            85103 non-null  object 
 4   Peak CCU                    85103 non-null  int64  
 5   Required age                85103 non-null  int64  
 6   Price                       85103 non-null  float64
 7   DLC count                   85103 non-null  int64  
 8   About the game              81536 non-null  object 
 9   Supported languages         85103 non-null  object 
 10  Full audio languages        85103 non-null  object 
 11  Reviews                     9743 non-null   object 
 12  Header image                85103 non-null  object 
 13  Website                     394

In [39]:
#Removing columns with either irrelevant or redundant data that will not help our study. Or any data that is not originally from steam
SteamGamesDroppedColumns = SteamGamesRaw.drop(['Header image','Metacritic url','Tags','Average playtime forever','Average playtime two weeks','Score rank','Metacritic score','User score','Median playtime forever','Median playtime two weeks','Screenshots','Movies'],axis=1,inplace=False).copy()
SteamGamesDroppedColumns 

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Linux,Positive,Negative,Achievements,Recommendations,Notes,Developers,Publishers,Categories,Genres
0,20200,Galactic Bowling,"Oct 21, 2008",0 - 20000,0,0,19.99,0,Galactic Bowling is an exaggerated and stylize...,['English'],...,False,6,11,30,0,NaN,Perpetual FX Creative,Perpetual FX Creative,"Single-player,Multi-player,Steam Achievements,...","Casual,Indie,Sports"
1,655370,Train Bandit,"Oct 12, 2017",0 - 20000,0,0,0.99,0,THE LAW!! Looks to be a showdown atop a train....,"['English', 'French', 'Italian', 'German', 'Sp...",...,False,53,5,12,0,NaN,Rusty Moyher,Wild Rooster,"Single-player,Steam Achievements,Full controll...","Action,Indie"
2,1732930,Jolt Project,"Nov 17, 2021",0 - 20000,0,0,4.99,0,Jolt Project: The army now has a new robotics ...,"['English', 'Portuguese - Brazil']",...,False,0,0,0,0,NaN,Campião Games,Campião Games,Single-player,"Action,Adventure,Indie,Strategy"
3,1355720,Henosis™,"Jul 23, 2020",0 - 20000,0,0,5.99,0,HENOSIS™ is a mysterious 2D Platform Puzzler w...,"['English', 'French', 'Italian', 'German', 'Sp...",...,True,3,0,0,0,NaN,Odd Critter Games,Odd Critter Games,"Single-player,Full controller support","Adventure,Casual,Indie"
4,1139950,Two Weeks in Painland,"Feb 3, 2020",0 - 20000,0,0,0.00,0,ABOUT THE GAME Play as a hacker who has arrang...,"['English', 'Spanish - Spain']",...,False,50,8,17,0,This Game may contain content not appropriate ...,Unusual Games,Unusual Games,"Single-player,Steam Achievements","Adventure,Indie"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85098,2669080,Mannerheim's Saloon Car,"Jan 2, 2024",0 - 0,0,0,0.00,0,Marshal Mannerheim’s Saloon Car is the train c...,"['English', 'Finnish']",...,False,0,0,0,0,NaN,Xamk Game Studios,"Sodan ja rauhan keskus Muisti, Päämajamuseo","Single-player,Tracked Controller Support,VR Only","Adventure,Simulation"
85099,2736910,Beer Run,"Jan 3, 2024",0 - 0,0,0,0.00,0,Beer Run is an Indie game created to steal bee...,['English'],...,False,0,0,0,0,NaN,955 Games,955 Games,Single-player,"Casual,Indie"
85100,2743220,My Friend The Spider,"Jan 4, 2024",0 - 0,0,0,0.00,0,A small 'horror' narrative game about isolatio...,['English'],...,False,0,0,0,0,NaN,MCA,MCA,Single-player,"Adventure,Simulation"
85101,2293130,Path of Survivors,"Jan 8, 2024",0 - 0,0,0,3.99,0,Path of Survivors is a multi-class auto-battle...,['English'],...,False,0,0,34,0,NaN,Limited Input,Limited Input,"Single-player,Steam Achievements,Partial Contr...","Action,Casual,Indie,RPG,Simulation"


In [40]:
# Removing rows with 0 peak concurrent users (peak ccu). This is to filter out most of the games with little to no attention, these typically have shotty information.
SteamGamesDroppedColumns = SteamGamesDroppedColumns[SteamGamesDroppedColumns['Peak CCU'] != 0]
SteamGamesDroppedColumns

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Linux,Positive,Negative,Achievements,Recommendations,Notes,Developers,Publishers,Categories,Genres
5,1469160,Wartune Reborn,"Feb 26, 2021",50000 - 100000,68,0,0.00,0,Feel tired of auto-fight? Feel tired of boring...,['English'],...,False,87,49,0,0,NaN,7Road,7Road,"Single-player,Multi-player,MMO,PvP,Online PvP,...","Adventure,Casual,Free to Play,Massively Multip..."
6,1659180,TD Worlds,"Jan 9, 2022",0 - 20000,3,0,10.99,1,"TD Worlds is a dynamic, highly strategical gam...","['English', 'Russian', 'Danish']",...,False,21,7,62,0,NaN,MAKSIM VOLKAU,MAKSIM VOLKAU,"Single-player,Steam Achievements,Steam Cloud","Indie,Strategy"
7,1968760,Legend of Rome - The Wrath of Mars,"May 5, 2022",0 - 20000,2,0,9.99,0,When the Roman people honored a simple warrior...,"['English', 'German']",...,False,0,0,0,0,NaN,magnussoft,magnussoft,"Single-player,Steam Cloud",Casual
8,1178150,MazM: Jekyll and Hyde,"Apr 2, 2020",0 - 20000,1,0,14.99,0,'MazM: Jekyll and Hyde' is a darkly entertaini...,"['English', 'French', 'Italian', 'German', 'Sp...",...,False,76,6,25,0,NaN,Growing Seeds,"CFK Co., Ltd.","Single-player,Steam Achievements,Full controll...","Adventure,RPG,Simulation,Strategy"
10,1026420,WARSAW,"Oct 2, 2019",20000 - 50000,5,0,23.99,0,Use everything at your disposal to help a team...,"['English', 'French', 'German', 'Polish', 'Rus...",...,False,589,212,34,427,NaN,Pixelated Milk,"Pixelated Milk,gaming company","Single-player,Steam Achievements,Steam Trading...","Indie,RPG"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85058,2738420,Backrooms: Eight Levels,"Jan 6, 2024",0 - 20000,3,0,0.74,0,An atmospheric horror game based on the popula...,['English'],...,False,1,0,0,0,NaN,Performance Artist,Performance Artist,Single-player,"Action,Adventure,Casual,Indie,RPG,Simulation"
85077,2704060,Ant Farm Simulator,"Jan 5, 2024",0 - 20000,3,0,0.99,0,Ant Farm (formicarium) With A Colony Of Ants. ...,"['English', 'Italian', 'German', 'Spanish - Sp...",...,False,1,1,0,0,NaN,Smirna Simulator Games,Smirna Simulator Games,Single-player,"Casual,Indie,Simulation,Early Access"
85084,2508400,Mothers and Daughters,"Jan 2, 2024",0 - 20000,29,0,8.49,0,"In Mothers and Daughters, you play as Max—an 1...",['English'],...,True,0,0,0,0,Mothers and Daughters is an Adult Visual Novel...,Spin256,Spin256,Single-player,"Adventure,Indie,Early Access"
85086,2000770,Ballance,"Jan 5, 2024",0 - 20000,34,0,3.59,0,Race your way to the finish line and overcome ...,"['English', 'French', 'Italian', 'German', 'Sp...",...,False,0,0,0,109,NaN,Cyparade,Ziggurat,Single-player,Casual


In [41]:
# Removing rows that don't contain Language information '[]' in 'Full audio languages' column, there is a strong correlation between games with no audio language information and having no players.
SteamGamesDroppedColumns = SteamGamesDroppedColumns[SteamGamesDroppedColumns['Full audio languages'] != '[]']
SteamGamesDroppedColumns

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Linux,Positive,Negative,Achievements,Recommendations,Notes,Developers,Publishers,Categories,Genres
7,1968760,Legend of Rome - The Wrath of Mars,"May 5, 2022",0 - 20000,2,0,9.99,0,When the Roman people honored a simple warrior...,"['English', 'German']",...,False,0,0,0,0,NaN,magnussoft,magnussoft,"Single-player,Steam Cloud",Casual
22,434030,Aerofly FS 2 Flight Simulator,"Nov 20, 2017",100000 - 200000,19,0,37.49,23,Aerofly FS 2 lets you explore the world of fly...,"['English', 'German', 'French']",...,True,1490,408,0,1831,NaN,IPACS,IPACS,"Single-player,Partial Controller Support","Action,Indie,Racing,Simulation"
39,262150,Vanguard Princess,"Mar 3, 2014",200000 - 500000,1,0,2.49,7,Government experiments on a young girl trigger...,['English'],...,True,2014,841,0,1477,NaN,Tomoaki Sugeno,eigoMANGA,"Single-player,Multi-player,PvP,Shared/Split Sc...","Action,Casual,Indie"
40,1192900,IRON REBELLION,"Nov 9, 2021",50000 - 100000,2,0,11.99,0,WHAT IS IRON REBELLION? Iron Rebellion is all ...,['English'],...,False,168,39,0,186,Iron Rebellion has elements of sci-fi combat w...,Black Beach Studio,Black Beach Studio,"Multi-player,PvP,Online PvP,Co-op,Online Co-op...","Action,Indie,Early Access"
42,231330,Deadfall Adventures,"Nov 15, 2013",100000 - 200000,4,0,19.99,0,Deadfall Adventures is an action-driven first-...,"['English', 'German', 'Polish', 'French', 'Rus...",...,True,1716,628,50,1140,NaN,The Farm 51,THQ Nordic,"Single-player,Multi-player,Co-op,Steam Achieve...","Action,Adventure"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84971,2707500,ROKO'S BASILISK,"Jan 5, 2024",0 - 20000,1,0,1.79,0,"'We have no idea what you'll find there, but w...","['English', 'Spanish - Spain', 'Portuguese - B...",...,False,9,0,3,0,NaN,Roko,Roko,"Single-player,Steam Achievements",Indie
84981,2710320,THE DESCENT,"Jan 5, 2024",0 - 20000,4,0,6.79,0,"Experience 'The Descent,' a first-person psych...","['English', 'French', 'Italian', 'German', 'Sp...",...,False,8,1,12,0,The game contains images of scenes of violence...,Celeritas Games,Celeritas Games,"Single-player,Steam Achievements","Indie,Simulation"
85015,2719650,Project XSTING,"Jan 5, 2024",0 - 20000,3,0,7.99,0,SUBJECT ON THE RUN! You’re tired of being a la...,"['English', 'Spanish - Spain', 'Portuguese - B...",...,False,5,0,42,0,NaN,Saucy Melon,Saucy Melon,"Single-player,Steam Achievements,Steam Cloud","Action,Casual,Indie,Early Access"
85034,2234690,The Day Before You Gone,"Jan 5, 2024",20000 - 50000,3,0,7.64,2,"Set in a medieval fantasy world, an era filled...","['English', 'Russian']",...,False,11,0,30,0,NaN,Aequalis Studio,Aequalis Studio,"Single-player,Steam Achievements","Adventure,Indie"


In [42]:
# Removing 'NaN' rows from the 'Categories' column, since they are less likely to provide any relevant data
SteamGamesDroppedColumns = SteamGamesDroppedColumns.dropna(subset=['Categories'])

In [43]:
# These upcoming lines of code will throw warnings, this should fix that
import warnings

warnings.filterwarnings('ignore')

In [44]:
# Taking a closer look at categories, going to see if I can't clean them up a bit
# Gonna convert the values to strings first. and make a copy
SteamGamesCategories = SteamGamesDroppedColumns.copy()
SteamGamesCategories['Categories'] = SteamGamesDroppedColumns['Categories'].astype(str)
# NOTICE -- This code will throw warnings, it does work though
categoryCounts = SteamGamesDroppedColumns['Categories'].str.split(',').explode().value_counts()
SteamGamesCategories['Categories'] = SteamGamesDroppedColumns['Categories'].str.split(',')
categoryCounts

Categories
Single-player                 10075
Steam Achievements             6576
Steam Cloud                    4509
Steam Trading Cards            3255
Full controller support        3161
Multi-player                   3121
PvP                            1798
Co-op                          1783
Partial Controller Support     1560
Online PvP                     1514
Online Co-op                   1221
Steam Leaderboards             1077
Remote Play Together           1062
Shared/Split Screen             903
Remote Play on TV               835
Shared/Split Screen PvP         627
Stats                           606
Cross-Platform Multiplayer      544
Shared/Split Screen Co-op       533
In-App Purchases                518
Steam Workshop                  510
Includes level editor           451
Remote Play on Tablet           440
Captions available              352
Remote Play on Phone            326
MMO                             279
LAN PvP                         137
LAN Co-op        

In [45]:
# NOTICE -- This code will throw warnings, it does work though
# Consolidating categories.

SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: ['Co-op' if 'co-op' in i.lower() else i for i in x])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: ['VR Support' if 'vr support' in i.lower() else i for i in x])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: ['Controller Support' if 'controller' in i.lower() else i for i in x])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: ['Cross-Platform' if 'cross-platform multiplayer' in i.lower() else i for i in x])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: ['Multi-player' if 'mmo' in i.lower() else i for i in x])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: ['Remote Play' if 'remote play' in i.lower() else i for i in x])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: ['PvP' if 'pvp' in i.lower() else i for i in x])

# Removing categories

SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'Mods'])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'Mods (require HL2)'])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'Includes level editor'])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'Commentary available'])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'Steam Turn Notifications'])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'Stats'])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'HDR available'])
SteamGamesCategories['Categories'] = SteamGamesCategories['Categories'].apply(lambda x: [i for i in x if i != 'Mods'])

In [46]:
# Final checks to see if the code did what I wanted it to do.
categoryCounts = SteamGamesCategories['Categories'].explode().value_counts()
categoryCounts

Categories
Single-player               10075
Steam Achievements           6576
Controller Support           4794
Steam Cloud                  4509
PvP                          4076
Co-op                        3673
Multi-player                 3400
Steam Trading Cards          3255
Remote Play                  2663
Steam Leaderboards           1077
Shared/Split Screen           903
Cross-Platform                544
In-App Purchases              518
Steam Workshop                510
Captions available            352
VR Only                        78
VR Support                     77
Valve Anti-Cheat enabled       75
SteamVR Collectibles           26
Includes Source SDK            19
Name: count, dtype: int64

In [47]:
# Checking genres for cleanliness. Looks A-Okay to me.
SteamGamesCategories['Genres'] = SteamGamesDroppedColumns['Genres'].str.split(',')
genreCounts = SteamGamesDroppedColumns['Genres'].str.split(',').explode().value_counts()
genreCounts

Genres
Indie                    6117
Adventure                4808
Action                   4700
Casual                   3291
Simulation               2489
Strategy                 2396
RPG                      2390
Early Access             1270
Free to Play              683
Sports                    534
Massively Multiplayer     413
Racing                    380
Violent                    31
Utilities                  18
Gore                       18
Video Production           10
Design & Illustration       8
Game Development            7
Sexual Content              6
Animation & Modeling        6
Education                   6
Software Training           5
Nudity                      4
Audio Production            3
Web Publishing              3
Photo Editing               1
Movie                       1
Name: count, dtype: int64

In [48]:
# Doing data time conversions
DateConversions = SteamGamesCategories.copy()
# Have to fix some formating first 🙄 some games do not have a set day
def add_day_if_missing(date_str):
    if len(date_str.split(' ')) == 2:
        return f'{date_str.split(" ")[0]} 1, {date_str.split(" ")[1]}'
    else:
        return date_str

DateConversions['Release date'] = DateConversions['Release date'].apply(add_day_if_missing)
# FINALLY we convert the date to something that computers can easily work with :D
DateConversions['Release date'] = pd.to_datetime(DateConversions['Release date'], format='%b %d, %Y')

DateConversions

,AppID,Name,Release date,Estimated owners,Peak CCU,Required age,Price,DLC count,About the game,Supported languages,...,Linux,Positive,Negative,Achievements,Recommendations,Notes,Developers,Publishers,Categories,Genres
7,1968760,Legend of Rome - The Wrath of Mars,2022-05-05,0 - 20000,2,0,9.99,0,When the Roman people honored a simple warrior...,"['English', 'German']",...,False,0,0,0,0,NaN,magnussoft,magnussoft,"[Single-player, Steam Cloud]",[Casual]
22,434030,Aerofly FS 2 Flight Simulator,2017-11-20,100000 - 200000,19,0,37.49,23,Aerofly FS 2 lets you explore the world of fly...,"['English', 'German', 'French']",...,True,1490,408,0,1831,NaN,IPACS,IPACS,"[Single-player, Controller Support]","[Action, Indie, Racing, Simulation]"
39,262150,Vanguard Princess,2014-03-03,200000 - 500000,1,0,2.49,7,Government experiments on a young girl trigger...,['English'],...,True,2014,841,0,1477,NaN,Tomoaki Sugeno,eigoMANGA,"[Single-player, Multi-player, PvP, PvP, Shared...","[Action, Casual, Indie]"
40,1192900,IRON REBELLION,2021-11-09,50000 - 100000,2,0,11.99,0,WHAT IS IRON REBELLION? Iron Rebellion is all ...,['English'],...,False,168,39,0,186,Iron Rebellion has elements of sci-fi combat w...,Black Beach Studio,Black Beach Studio,"[Multi-player, PvP, PvP, Co-op, Co-op, Cross-P...","[Action, Indie, Early Access]"
42,231330,Deadfall Adventures,2013-11-15,100000 - 200000,4,0,19.99,0,Deadfall Adventures is an action-driven first-...,"['English', 'German', 'Polish', 'French', 'Rus...",...,True,1716,628,50,1140,NaN,The Farm 51,THQ Nordic,"[Single-player, Multi-player, Co-op, Steam Ach...","[Action, Adventure]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84971,2707500,ROKO'S BASILISK,2024-01-05,0 - 20000,1,0,1.79,0,"'We have no idea what you'll find there, but w...","['English', 'Spanish - Spain', 'Portuguese - B...",...,False,9,0,3,0,NaN,Roko,Roko,"[Single-player, Steam Achievements]",[Indie]
84981,2710320,THE DESCENT,2024-01-05,0 - 20000,4,0,6.79,0,"Experience 'The Descent,' a first-person psych...","['English', 'French', 'Italian', 'German', 'Sp...",...,False,8,1,12,0,The game contains images of scenes of violence...,Celeritas Games,Celeritas Games,"[Single-player, Steam Achievements]","[Indie, Simulation]"
85015,2719650,Project XSTING,2024-01-05,0 - 20000,3,0,7.99,0,SUBJECT ON THE RUN! You’re tired of being a la...,"['English', 'Spanish - Spain', 'Portuguese - B...",...,False,5,0,42,0,NaN,Saucy Melon,Saucy Melon,"[Single-player, Steam Achievements, Steam Cloud]","[Action, Casual, Indie, Early Access]"
85034,2234690,The Day Before You Gone,2024-01-05,20000 - 50000,3,0,7.64,2,"Set in a medieval fantasy world, an era filled...","['English', 'Russian']",...,False,11,0,30,0,NaN,Aequalis Studio,Aequalis Studio,"[Single-player, Steam Achievements]","[Adventure, Indie]"


In [49]:
# I believe this data set is now CLEAN! 🥳🥳🥳🥳
SteamGamesClean = DateConversions.copy()

# Create Dataframes for each Table

In [50]:
app_desc_df = SteamGamesClean.loc[:,['AppID', 'Name', 'About the game']]
app_desc_df.head(5)

,AppID,Name,About the game
7,1968760,Legend of Rome - The Wrath of Mars,When the Roman people honored a simple warrior...
22,434030,Aerofly FS 2 Flight Simulator,Aerofly FS 2 lets you explore the world of fly...
39,262150,Vanguard Princess,Government experiments on a young girl trigger...
40,1192900,IRON REBELLION,WHAT IS IRON REBELLION? Iron Rebellion is all ...
42,231330,Deadfall Adventures,Deadfall Adventures is an action-driven first-...


In [51]:
contact_df = SteamGamesClean.loc[:,['AppID', 'Support url', 'Website', 'Support email']]
contact_df.head(5)

,AppID,Support url,Website,Support email
7,1968760,http://magnussoft.biz/,http://magnussoft.biz/,support@magnussoft.de
22,434030,https://www.aerofly.com/support,https://www.aerofly.com/,NaN
39,262150,http://www.eigoMANGA.com/,http://www.eigoMANGA.com/vanguard,business@eigomanga.com
40,1192900,iron-rebellion.com,https://www.iron-rebellion.com/,ironrainvr@gmail.com
42,231330,NaN,http://www.deadfall-game.com/,support@quanticlab.com


In [52]:
app_review_df = SteamGamesClean.loc[:,['AppID', 'Recommendations', 'Notes', 'Positive', 'Negative']]
app_review_df.head(5)

,AppID,Recommendations,Notes,Positive,Negative
7,1968760,0,NaN,0,0
22,434030,1831,NaN,1490,408
39,262150,1477,NaN,2014,841
40,1192900,186,Iron Rebellion has elements of sci-fi combat w...,168,39
42,231330,1140,NaN,1716,628


In [53]:
app_detail_df = SteamGamesClean.loc[:,['AppID', 'Release date', 'Price', 'Required age', 'Windows', 'Mac', 'Linux']]
app_detail_df.head(5)

,AppID,Release date,Price,Required age,Windows,Mac,Linux
7,1968760,2022-05-05,9.99,0,True,False,False
22,434030,2017-11-20,37.49,0,True,True,True
39,262150,2014-03-03,2.49,0,True,True,True
40,1192900,2021-11-09,11.99,0,True,False,False
42,231330,2013-11-15,19.99,0,True,False,True


# Get Postgre Database Connection

In [55]:
# Starting PosgreSQL import 
# NOTICE -- BE SURE TO CONFIGURE 'PostgreSQL.py' BEFORE CONTINUING
import psycopg2
from psycopg2 import Error
from sqlalchemy import create_engine

# Create a database connection
engine = create_engine(PostgreSQL.connectionString)
# Create a local database connection
#engine = create_engine(connectionStringLocal)

# Create Tables in Database

In [56]:
app_desc_df.to_sql("app_description", con = engine, if_exists = 'replace', index = False)

594

In [57]:
contact_df.to_sql("contact", con = engine, if_exists = 'replace', index = False)

594

In [58]:
app_review_df.to_sql("app_review", con = engine, if_exists = 'replace', index = False)

594

In [59]:
app_detail_df.to_sql("app_detail", con = engine, if_exists = 'replace', index = False)

594